# Gausian Naive Bayes On Full Dataset, Applyin PCA

In [1]:
import numpy as np

import pandas as pd

import scipy.stats as s

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
data = pd.read_csv("data.csv")

data.drop([data.columns[0], data.columns[32]], axis =1, inplace = True)

data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
labels = np.array(data['diagnosis']).reshape(data['diagnosis'].shape[0],1)

X = np.array(data.iloc[:,1:])

X.shape

(569, 30)

In [4]:
mu = np.mean(X, axis=0)

mu = mu.reshape(-1,mu.shape[0])

X_dash  = X - mu

sigma_hat = (1/data.shape[0])*np.matmul(X_dash.T,X_dash)

sigma_hat_decomposed = np.linalg.svd(sigma_hat)

Q = sigma_hat_decomposed[0]

lambdaa = sigma_hat_decomposed[1]

Q_tilda = Q[:,0:15]

X_new = np.matmul(X_dash, Q_tilda)

In [5]:
new_data = pd.DataFrame(data=X_new)

new_data['diagnosis'] = labels

In [6]:
training_data_len = int(0.7*new_data.shape[0])

benign_training_data = new_data[new_data['diagnosis'] == 'B'].iloc[0:training_data_len//2]

malignant_training_data = new_data[new_data['diagnosis'] == 'M'].iloc[0:training_data_len//2]

training_data = pd.concat([benign_training_data, malignant_training_data])

In [7]:
cv_data_len = int(0.2 * new_data.shape[0])

begning_remaining_data = new_data[new_data['diagnosis'] == 'B'].iloc[training_data_len//2:]

malignant_remaining_data = new_data[new_data['diagnosis'] == 'M'].iloc[training_data_len//2:]

remaining_data = pd.concat([begning_remaining_data,malignant_remaining_data])

cv_data = remaining_data.iloc[0:cv_data_len]

testing_data = remaining_data.iloc[cv_data_len:]

In [8]:
mu_hat_m = np.array(training_data[training_data['diagnosis'] == 'M'].iloc[:,0:15].mean())

sigma_hat_m = np.array(training_data[training_data['diagnosis'] == 'M'].iloc[:,0:15].cov()) 

np.linalg.det(sigma_hat_m)

malignant_prior = training_data[training_data['diagnosis'] == 'M'].shape[0]/training_data.shape[0]

In [9]:
mu_hat_b = np.array(training_data[training_data['diagnosis'] == 'B'].iloc[:,0:15].mean())

sigma_hat_b = np.array(training_data[training_data['diagnosis'] == 'B'].iloc[:,0:15].cov())

np.linalg.det(sigma_hat_b)

benign_prior = training_data[training_data['diagnosis'] == 'B'].shape[0]/training_data.shape[0]

In [10]:
def cv_data_testing(data):
    
    inputs = np.array(data.iloc[:,0:15])
    
    posterior_m = s.multivariate_normal.pdf(inputs, mu_hat_m, sigma_hat_m) * malignant_prior
    
    posterior_b = s.multivariate_normal.pdf(inputs, mu_hat_b, sigma_hat_b) * benign_prior
    
    boolean_mask = posterior_m > posterior_b
    
    predicted_category = pd.Series(boolean_mask)
    
    predicted_category.replace(to_replace=[False, True], value = ['B', 'M'], inplace=True)
    
    return np.array(predicted_category)

In [11]:
cv_results = cv_data_testing(cv_data)

actual_results = np.array(cv_data['diagnosis'])

boolean_mask = cv_results == actual_results

cv_accuracy = np.count_nonzero(boolean_mask)/boolean_mask.shape[0]

cv_accuracy

0.9292035398230089

In [12]:
testing_results = cv_data_testing(testing_data)

actual_testing_results = np.array(testing_data['diagnosis'])

testing_accuracy = np.count_nonzero(testing_results == actual_testing_results)/actual_testing_results.shape[0]

testing_accuracy

0.9827586206896551

In [13]:
testing_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,diagnosis
510,468.335643,15.365047,-0.827115,-6.946220,3.742001,1.082143,-0.513718,0.284520,-0.423439,-0.175653,-0.080218,-0.019155,0.006649,0.042927,-0.001553,B
511,90.547058,84.702937,-14.862191,-7.320504,4.702856,0.650109,-0.434868,0.288813,0.043768,0.031598,0.195911,-0.082660,-0.001257,-0.005652,-0.005064,B
513,13.882992,13.242216,-2.582961,-4.770083,9.141309,0.684482,-0.042296,0.294719,0.305757,0.046843,-0.039884,-0.014032,-0.044262,0.006641,0.013791,B
515,482.149027,-17.043363,5.320532,-3.439970,-1.866508,1.191039,-0.647507,0.337653,-0.106263,-0.050447,-0.049251,0.034747,-0.003650,-0.044215,-0.011949,B
518,260.052625,-31.315042,8.068103,4.770763,5.280829,1.039662,0.152870,-0.179798,0.157245,-0.161032,0.099249,0.135555,0.043911,0.073074,0.024602,B
519,303.492556,-4.545996,5.079922,-0.364805,4.307432,0.739271,0.022233,0.062852,0.192189,-0.075811,0.030442,0.068702,0.021684,-0.024520,0.016075,B
520,681.583641,-52.401063,18.694701,-8.558976,1.882810,-1.458548,-0.463803,-0.413878,-0.028669,-0.232312,-0.162230,0.133028,0.067835,-0.055290,0.008308,B
522,520.808520,2.302135,-1.453325,-2.671834,-6.084091,0.140763,0.117194,0.404510,-0.152752,-0.092797,0.200642,0.027405,-0.009854,-0.013457,-0.005394,B
523,196.893522,21.103080,-3.428177,2.166690,1.220961,1.014706,0.752999,-0.026321,-0.037383,-0.038738,0.132185,-0.025460,0.047973,0.009441,0.022213,B
524,620.561204,-48.093794,10.372840,-3.537066,-0.189877,-2.813347,0.648383,-0.075347,-0.038891,-0.082607,0.036697,0.083244,0.018978,0.073755,0.010715,B
